# Chapter 6 -- Higher Order Functions


In [1]:
-- IHaskell settings
:opt no-lint
:opt no-pager

**Higher order function**: a function that does either of:
- take functions as parameters
- return functions as return value

> Higher order functions aren't just a part of the Haskell experience, they pretty much are _the Haskell experience_.
> It turns out that if you want to define computations by defining what stuff _is_ instead of defining steps that change some state and maybe looping them, higher order functions are indispensable.
> They're a really powerful way of solving problems and thinking about programs.



## Curried Functions

... Every function in Haskell officially only takes one parameter; so how is it possible that we defined and used several function that take more than one parameter so far ?

The clever trick: all the functions that accepted _several parameters_ so far have been **Curried Functions**

Example: `max 4 5`:
- first creates a function that takes a parameter and return either 4 or that parameter, depending on which is bigger
- then, `5` is applied to that function and that function produced our desired result
- ... sounds mouthful, but it's actually a really cool concept


In [2]:
-- So the following two calls are equivalent
max 4 5
(max 4) 5
:t max
:t max 4

5

5

max :: forall a. Ord a => a -> a -> a

max 4 :: forall a. (Ord a, Num a) => a -> a

The space is sort of like an operator (function application) and it has the highest precedence; `max :: (Ord a) => a -> a -> a` can be written as `max :: (Ord a) => a -> (a -> a)`:
- read as `max` takes an `a` and returns (that's the `->`) a function that takes an `a` and returns an `a`
- this is why the return type and the parameters of functions are all simply separated with arrows

How beneficial ?:
- **partially applied function**: a function that takes as many parameters as we left out
- using partial application (i.e. calling functions with too few parameters, if we will) is a neat way to create functions on the fly so we can pass them to another function or to seed them with some data


Verbose explanation


In [3]:
multThree :: (Num a) => a -> a -> a -> a
multThree x y z = x * y * z

multThree 3 5 9
((multThree 3) 5) 9

135

135

`multThree 3 5 9` (or `((multThree 3) 5) 9`):

1. first, `3` is applied to `multThree`, because they're separated by a space; that creates a function that takes one parameter and returns a function
2. then `5` is applied to that, which creates a function that will take a parameter and multiply it by `15`
3. finally `9` is applied to that function and the result is `135` or something

Function type `multThree :: (Num a) => a -> (a -> (a -> a))`
(the thing before the `->` is the parameter that a function takes and the thing after it is what it returns):

1. first, `multThree` takes an `a` and returns a function of type `(Num a) => a -> (a -> a)`
2. then this function takes an `a` and returns function of type `(Num a) => a -> a`
3. finally this function just takes an `a` and return an `a`


In [4]:
let multTwoWithNine = multThree 9
multTwoWithNine 2 3
let multWithEighteen = multTwoWithNine 2
multWithEighteen 10

54

180

In [5]:
compareWithHundred :: (Num a, Ord a) => a -> Ordering
compareWithHundred x = compare 100 x
compareWithHundred 99

compareWithHundred :: (Num a, Ord a) => a -> Ordering
compareWithHundred = compare 100 -- wow !
compareWithHundred 99

GT

GT

By calling functions with too few parameters, so to speak, we're creating new functions _on the fly_.


Infix functions can also be partially applied by using sections:
- to section an infix function, simply surround it with parentheses and only supply a parameter on one side
- that creates a function that takes one parameter and then applies it to the side that's missing an operand


In [6]:
devideByTen :: (Floating a) => a -> a
devideByTen = (/10)

devideTen :: (Floating a) => a -> a
devideTen = (10/)

isUpperAlphanum :: Char -> Bool
isUpperAlphanum = (`elem` ['A'..'Z'])

devideTen 5
devideByTen 5
isUpperAlphanum 's'
isUpperAlphanum 'S'

2.0

0.5

False

True

In [7]:
-- NOTE:
-- The only special thing about sections is using `-`;
-- for convenience, (-4) means minus four, not a partially applied function.
-- For that, we need partially apply the `subtract` function
subtractFour = subtract 4
subtractFour 10

6

NOTE: Functions aren't instances of the `Show` typeclass, so we can't get a neat string representation of a function; so the error below


In [8]:
multThree 3 4

: 

## Higher-orderism

Functions can take functions as parameters and also return functions.


In [9]:
applyTwice :: (a -> a) -> a -> a
applyTwice f x = f (f x)

Type declaration: `applyTwice :: (a -> a) -> a -> a`:
- the parentheses are mandatory: that indicates the first parameter is a function that takes something and returns the same thing
  * NOTE: parentheses are right-associative
- let's read as:
  * this function takes two parameters and returns one thing
  * the first parameter is a function (of type `a -> a`)
  * the second is that same `a`

Body itself is so simple; we can understand it well


In [10]:
applyTwice (+3) 10
applyTwice (++ " HAHA") "HEY"
applyTwice ("HAHA " ++) "HEY"
applyTwice (multThree 2 2) 9
applyTwice (1:) [3] -- here we go !

16

"HEY HAHA HAHA"

"HAHA HAHA HEY"

144

[1,1,3]

`zipWith`: takes a function and two lists as parameters and then joins the two lists by applying the function between corresponding elements


In [11]:
zipWith' :: (a -> b -> c) -> [a] -> [b] -> [c]
zipWith' _ [] _          = []
zipWith' _ _ []          = []
zipWith' f (x:xs) (y:ys) = f x y : zipWith' f xs ys

zipWith' (+) [1..4] [(-1),(-2)..(-4)]
zipWith' max [1..4] [(-1),(-2)..(-4)]
zipWith' (++) ["foo ", "bar ", "baz "] ["fighters", "hoppers", "aldrin"]
zipWith' (+) (replicate 5 2) [1..]
zipWith' (zipWith' (+)) [[1..3], [4..6], [7..9]] [[1..3], [4..6], [7..9]]

[0,0,0,0]

[1,2,3,4]

["foo fighters","bar hoppers","baz aldrin"]

[3,4,5,6,7]

[[2,4,6],[8,10,12],[14,16,18]]

`flip`: takes a function and returns a function that is like our original function, only the first two arguments are flipped


In [12]:
flip' :: (a -> b -> c) -> (b -> a -> c) -- the second parentheses aren't really necessary
flip' f = g
  where g x y = f y x

By taking advantage of the fact that functions are curried, we can defined it in an even more simpler manner:


In [13]:
-- NOTE: when we call `flip' f` without the parameters `y` and `x`, it will return an `f` that takes those two parameters but calls them flipped.
flip' :: (a -> b -> c) -> b -> a -> c
flip' f y x = f x y

flip' zip [1..5] "hello"
zipWith (flip' div) [2,2..] [10,8..2]

[('h',1),('e',2),('l',3),('l',4),('o',5)]

[5,4,3,2,1]

## `map`s and `filter`s

> Mapping and filtering is the bread and butter of every functional programmer's toolbox.


`map`: one of those really versatile higher-order functions that can be used in millions of different ways


In [14]:
map' :: (a -> b) -> [a] -> [b]
map' _ []     = []
map' f (x:xs) = f x : map f xs

map' (+3) [1,5,3,1,6]
map' (++ "!") ["BIFF", "BANG", "POW"]
map' (replicate 3) [3..6]
map' (map' (^2)) [[1,2], [3..6], [7,8]]
map' fst [(1,2), (3,4), (5,6)]

[4,8,6,4,9]

["BIFF!","BANG!","POW!"]

[[3,3,3],[4,4,4],[5,5,5],[6,6,6]]

[[1,4],[9,16,25,36],[49,64]]

[1,3,5]

`filter`:

In [15]:
filter' :: (a -> Bool) -> [a] -> [a]
filter' _ [] = []
filter' p (x:xs)
  | p x       = x : filter' p xs
  | otherwise = filter' p xs

filter' (>3) [1..10]
filter' (==3) [1..5]
filter' even [1..10]
let notNull x = not (null x) in filter notNull [[1..3], [], [4..6], [2,2], [], []]
filter (`elem` ['a'..'z']) "u LaUgH aT mE BeCaUsE I aM diFfeRent"
filter (`elem` ['A'..'Z']) "i lauGh At You BecAuse u r aLL the Same"

[4,5,6,7,8,9,10]

[3]

[2,4,6,8,10]

[[1,2,3],[4,5,6],[2,2]]

"uagameasadifeent"

"GAYBALLS"

NOTE: the same thing could be achieved with a list comprehension:
- there's no set rule for when to use `map` and `filter` versus using list comprehension; we just have to decide what's more readable depending on the code and the context


In [16]:
quicksort :: (Ord a) => [a] -> [a]
quicksort [] = []
quicksort (x:xs) =
  smallerSorted ++ [x] ++ biggerSorted
  where smallerSorted = quicksort (filter (<=x) xs)
        biggerSorted  = quicksort (filter (>x) xs)

quicksort "Shuhei Kadowaki"

" KSaadehhiikouw"

Haskell laziness & `map`/`filter`:
> Thanks to Haskell's laziness, even if you map something over a list several times and filter it several times, it will only pass over the list once.


**find the largest number under 100,000 that's divisible by 3829**


In [17]:
largestDivisible :: (Integral a) => a
largestDivisible = head (filter p [100000,99999..]) -- this can be infinite list; the computation stops anyways
  where p x = x `mod` 3829 == 0

**find the sum of all odd squares that are smaller than 10,000**

`takeWhile`:
- takes a predicate and a list and then goes from the beginning of the list and returns its elements while the predicate holds true; once an element is found for which the predicate doesn't hold, it stops


In [18]:
-- awesome !
sum (takeWhile (<10000) (filter odd (map (^2) [1..])))

-- or using list comprehension (imho less readable)
sum (takeWhile (<10000) [ m | m <- [ n^2 | n <- [1..] ], odd m])

166650

166650

What makes this possible ? Haskell's property of laziness !:
- we can `map` over and `filter` an infinite list, because it won't actually `map` and `filter` it right away; it will delay those actions
- only when we force Haskell to show us the sum, the `sum` function say to the `takeWhile` that it needs those numbers
- `takeWhile` forces the `filter`ing and `map`ping to occur, but only until a number greater than or equal to 10,000 is encountered


**for all starting numbers between 1 and 100, how many Collatz chains have a length greater than 15 ?**

NOTE: Collatz chain always finishes at the number 1, for all starting numbers


In [19]:
chain :: (Integral a) => a -> [a]
chain 1 = [1]
chain n
  | even n = n : chain (n `div` 2)
  | odd n  = n : chain (n * 3 + 1)

chain 10

length (filter (>15) (map length (map chain [1..100])))

[10,5,16,8,4,2,1]

66

more higher-order function: `map (*) [0..]`
- c.f.: so far, we've only mapped functions that take one parameter over lists: e.g.: `map (*2) [0..]`
- what happens here: the number in the list is applied to the function `*`, which has a type of `(Num a) => a -> a -> a`
  * applying only one parameter to a function that takes two parameters returns a partially applied function, which takes one parameter
  * so we get back _a list of functions_ that only take one parameter, so `(Num a) => [a -> a]`
    + `map (*) [0..]` produces a list like `[(*0), (*1), ...]`


In [20]:
let listOfFuns = map (*) [0..] in (listOfFuns !! 5) 2

10

## Lambdas

Lambdas: anonymous functions that are used because we need some functions only once; normally passed into a higher-order function


In [21]:
-- syntax: `(\args -> body)`
zipWith (\a b -> (a * 30 + 3) / b) [5,4..1] [1..5]

-- caveat: don't abuse lambdas; using partial functions can lead to more readable code
map (+1) [0..9]
map (\x -> x + 1) [0..9]

-- pattern match: cant defined fall back definition !
map (\(a,b) -> a + b) [(1,2),(3,5),(6,3),(2,6)]

-- -- without parentheses, they extend all the way to the right
addThree :: (Num a) => a -> a -> a -> a
addThree x y z = x + y + z

-- addThree :: (Num a) => a -> a -> a -> a
-- addThree = \x -> \y -> \z -> x + y + z

-- more "sensible" `flip'` definition: it would be clearer that `flip'` is used for producing a new function most of the time
flip' :: (a -> b -> c) -> b -> a -> c
flip' f = \x y -> f y x

flip' (++) "kadowaki" "shuhei"

[153.0,61.5,31.0,15.75,6.6]

[1,2,3,4,5,6,7,8,9,10]

[1,2,3,4,5,6,7,8,9,10]

[3,8,9,8]

"shuheikadowaki"

## `fold`s

`fold` encapsulate the common pattern of the recursive functions on lists:
- have an edge case for the empty list
- introduce `x:xs` pattern and then do some action that involves a signle element and the rest of the list

> Folds can be used to implement any function where you traverse a list once, element by element, and then return something based on that. Whenever you want to traverse a list to return something, chances are you want a `fold`.


`foldl f acc xs`
- `f acc x`: binary function: produces a new accumulator from the current accumulator `acc` and the current list head `x`
- `acc`: a starting value, a.k.a. accumulator
- `list`: list to fold up


In [22]:
sum' :: (Num a) => [a] -> a
sum' xs = foldl (\acc x -> acc + x) 0 xs

sum' [1..10]

55

In [23]:
-- even more succinctly
sum' :: (Num a) => [a] -> a
sum' = foldl (+) 0 -- carrying !

sum' [1..10]

55

In [24]:
elem' :: (Eq a) => a -> [a] -> Bool
elem' y ys = foldl (\acc x -> if x == y then True else acc) False ys

elem' 1 [0..10]
-- elem' 1 [0..] -- TODO

True

`foldr f acc xs`
- `f x acc`: binary function: produces a new accumulator from the current accumulator `acc` and the current list head `x`
- `acc`: a starting value, a.k.a. accumulator
- `list`: list to fold up


In [25]:
map' :: (a -> b) -> [a] -> [b]
map' f xs = foldr (\x acc -> f x : acc) [] xs
-- map' f xs = foldl (\acc x -> acc ++ [f x]) [] xs -- NOTE: `++` is much more expensive than `:`

map' (+1) [0..9]

[1,2,3,4,5,6,7,8,9,10]

big difference:
- `foldr` can work on infinite lists, i.e. take an infinite list _at some point_ should work with `foldr`
- `foldl` can't on infinite lists


`foldl1`, `foldr1`:
- don't require an explicit starting value
- can be used when a function doesn't make sense given an empty list


In [26]:
maximum' :: (Ord a) => [a] -> a
maximum' = foldr1 (\x acc -> if x > acc then x else acc)

reverse' :: [a] -> [a]
reverse' = foldl (\acc x -> x : acc) [] -- or `foldl (flip (:)) []`

product' :: (Num a) => [a] -> a
product' = foldr1 (*)

filter' :: (a -> Bool) -> [a] -> [a]
filter' p = foldr (\x acc -> if p x then x : acc else acc) []

head' :: [a] -> a
head' = foldr1 (\x _ -> x)

last' :: [a] -> a
last' = foldr1 (\_ x -> x)

`scanl`, `scanr`, `scanl1`, `scanr1`: report all the intermediate accumulator states in the form of a list


In [27]:
scanl (+) 0 [1..10]
scanr (+) 0 [1..10]
scanl1 (\acc x -> if x > acc then x else acc) [3,4,5,3,7,9,2,1]
scanl (flip (:)) [] [3,2..1]

[0,1,3,6,10,15,21,28,36,45,55]

[55,54,52,49,45,40,34,27,19,10,0]

[3,4,5,5,7,9,9,9]

[[],[3],[2,3],[1,2,3]]

**How many elements does it take for the sum of the roots of all natural numbers to exceed 1000 ?**


In [28]:
sqrtSum = length (takeWhile (<1000) (scanl1 (+) (map sqrt [1..])))
sum (map sqrt [1..130])
sum (map sqrt [1..131])

993.6486803921487

1005.0942035344083

## Function application with `$`

```haskell
($) :: (a -> b) -> a -> b
f $ x = f x
```
- normal function application (putting a space between two things): high precedence, left-associative
- `$` function: lowest precedence, right-associative

So, how this seemingly useless operator can be useful ?:
- we don't have to write so many parentheses !
  * `sum (map sqrt [1..130])` ⟺ `sum $ map sqrt [1..130]`
  * `sqrt (3 + 4 + 9)` ⟺ `sqrt $ 3 + 4 + 9`
  * `sum (filter (>10) (map (*2) [2..10]))` ⟺ `sum $ filter (>10) $ map (*2) [2..10]`
- the function application can be treated just like another function
  * e.g. map function application _over a list of functions_: `map ($ 3) [(4+), (10*), (^2), sqrt]`


In [29]:
sum $ filter (>10) $ map (*2) [2..10]
map ($ 3) [(4+), (10*), (^2), sqrt]

80

[7.0,30.0,9.0,1.7320508075688772]

## Function composition

$(f \circ g)(x) = f(g(x))$:
```haskell
(.) :: (b -> c) -> (a -> b) -> a -> c
f . g = \x -> f (g x)
```

NOTE: function composition can make functions on the fly as lambdas do, but many times, is clearer and more concise


In [30]:
map (\x -> negate (abs x)) [5,-3,-6,7,-3,2,-19,24]
map (negate . abs) [5,-3,-6,7,-3,2,-19,24]

[-5,-3,-6,-7,-3,-2,-19,-24]

[-5,-3,-6,-7,-3,-2,-19,-24]

Function composition is right-associative


In [31]:
map (\xs -> negate (sum (tail xs))) [[1..5], [3..6], [1..7]]
map (negate . sum . tail) [[1..5], [3..6], [1..7]]

[-14,-15,-27]

[-14,-15,-27]

When functions take several parameters: we usually have to partially apply them just so much that each function takes just one parameter
- the expressions below are all equivalent:
  * `sum (replicate 5 (max 6.7 8.9))`
  * `(sum . replicate 5 . max 6.7) 8.9`
  * `sum . replicate 5 . max 6.7 $ 8.9`

In [32]:
replicate 3 (product (map (*3) (zipWith max [1..5] [6..10])))
replicate 3 . product . map (*3) . zipWith max [1..5] $ [6..10]

[7348320,7348320,7348320]

[7348320,7348320,7348320]

"point free style" (or "pointless style") function definition:
```haskell
sum' :: (Num a) => [a] -> a
sum' - foldr (+) 0 -- no `xs` on both side
```

```haskell
fn x = ceiling (negate (tan (cos (max 50 x))))
```
can be defined
```haskell
fn = ceiling . negate . tan . cos . max 50
```

caveat: a huge composition chain can be less readable, which compositing a long chain can be happy for programmers ...
- `oddSquareSum = sum (takeWhile (<10000) (filter odd (map (^2) [1..])))`: agh ...
- `oddSquareSum = sum . takeWhile (<10000) . filter odd . map (^2) $ [1..]`: another agh ...
- ⟹ use `let` (when there's a chance of someone else reading this)


In [33]:
oddSquareSum =
  let oddSquares = filter odd $ map (^2) [1..]
      belowLimit = takeWhile (<10000) oddSquares
  in sum belowLimit